# Supply Chain Network Optimization

In this project, we are managing a supply chain network with the goal of minimizing the overall cost to meet customer demand. The network comprises five suppliers, each providing four types of raw materials. These raw materials are utilized by three factories to manufacture four distinct products. The output from these three factories is then distributed to fulfill the demand of four customers across the product range.

For optimization purposes, we consider all three factories as part of a single ownership entity. The objective is to determine the optimal solution that dictates the quantity of raw materials to be ordered from suppliers and the corresponding delivery of final products from the factory to each customer. The aim is to achieve cost efficiency throughout the entire supply chain process.


In [1]:
!pip install ortools

                                              0.0/101.1 MB ? eta -:--:--
                                              0.1/101.1 MB 2.6 MB/s eta 0:00:39
                                              0.3/101.1 MB 3.4 MB/s eta 0:00:30
                                              0.5/101.1 MB 4.1 MB/s eta 0:00:25
                                              0.8/101.1 MB 4.8 MB/s eta 0:00:21
                                              1.1/101.1 MB 5.0 MB/s eta 0:00:20
                                              1.4/101.1 MB 5.2 MB/s eta 0:00:20
                                              1.7/101.1 MB 5.6 MB/s eta 0:00:18
                                              1.9/101.1 MB 5.6 MB/s eta 0:00:18
                                              2.1/101.1 MB 5.7 MB/s eta 0:00:18
                                              2.1/101.1 MB 5.7 MB/s eta 0:00:18
     -                                        2.6/101.1 MB 5.6 MB/s eta 0:00:18
     -                                        2

In [2]:
import pandas as pd
import numpy as np
from ortools.linear_solver import pywraplp

### 1. Loading the input data 

In [3]:

sup_stock = pd.read_excel("Data_sc_network_optimization.xlsx", sheet_name = "Supplier stock", index_col=0)
raw_material_cost = pd.read_excel("Data_sc_network_optimization.xlsx", sheet_name = "Raw material costs", index_col=0)
raw_material_shipping = pd.read_excel("Data_sc_network_optimization.xlsx", sheet_name = "Raw material shipping", index_col=0)
production_req = pd.read_excel("Data_sc_network_optimization.xlsx", sheet_name = "Product requirements", index_col=0)
production_capacity = pd.read_excel("Data_sc_network_optimization.xlsx", sheet_name = "Production capacity", index_col=0)
customer_demand = pd.read_excel("Data_sc_network_optimization.xlsx", sheet_name = "Customer demand", index_col=0)
production_cost = pd.read_excel("Data_sc_network_optimization.xlsx", sheet_name = "Production cost", index_col=0)
shipping_costs = pd.read_excel("Data_sc_network_optimization.xlsx", sheet_name = "Shipping costs", index_col=0)

In [4]:
dataframes = [customer_demand, production_req, sup_stock, production_capacity, raw_material_cost, production_cost]

for df in dataframes:
    df.fillna(0, inplace=True)

Getting list of factories 

In [5]:


factories = list(raw_material_shipping.columns )
print("Factories:\n",factories)

# Getting list of materials
materials = list(raw_material_cost.columns)
print("Materials: \n",materials)

# Getting list of suppliers
suppliers = list(raw_material_cost.index)
print("Suppliers: \n",suppliers)

#Getting list of products
products = list(production_req.index)
print("Products: \n",products)

#Getting list of customers
customers = list(customer_demand.columns)
print("Customers: \n",customers)


Factories:
 ['Factory A', 'Factory B', 'Factory C']
Materials: 
 ['Material A', 'Material B', 'Material C', 'Material D']
Suppliers: 
 ['Supplier A', 'Supplier B', 'Supplier C', 'Supplier D', 'Supplier E']
Products: 
 ['Product A', 'Product B', 'Product C', 'Product D']
Customers: 
 ['Customer A', 'Customer B', 'Customer C', 'Customer D']


### 2. Creating ortools solver to solve integer programming (SCIP used to solve mixed integer linear programming)

In [6]:
solver = pywraplp.Solver.CreateSolver('SCIP')
# Adding decision variable to solver with name ex-Factory A_Materiaal A_Supplier A, total 60 variable

orders = {}
for factory in factories:
    for material in materials:
        for supplier in suppliers:
            orders[(factory, material, supplier)] = solver.IntVar(0, solver.infinity(), 
                                          factory+"_"+material+"_"+supplier)

# decision vriable for production volume, total 12 variable
production_volume = {}
for factory in factories:
    for product in products: 
        production_volume[(factory, product)] = solver.IntVar(0, solver.infinity()  , factory+"_"+product)

# Decision variable for delivery quantity, total 48 variable
delivery = {}
for factory in factories: 
    for customer in customers:
        for product in products: 
            delivery[(factory, customer, product)] = solver.IntVar(0, solver.infinity(), factory+"_"+customer+"_"+product)


In [7]:
# Defining the constraints that ensure factories produce more than they ship to the customers

for product in products: 
    for factory in factories:
        c = solver.Constraint(0, solver.infinity())
        c.SetCoefficient(production_volume[(factory, product)] , 1)
        for customer in customers:             
            c.SetCoefficient(delivery[(factory, customer, product)], -1)


# Defining the constraints that ensure that customer demand is met(delivery is greater tha or equal to customer demand)

for customer in customers: 
    for product in products:
        
        c = solver.Constraint(int(customer_demand.loc[product][customer]),solver.infinity())
        for factory in factories: 
            c.SetCoefficient(delivery[(factory,customer,product)], 1)

# Defining the constraints that ensure that suppliers have all ordered items in stock     

for supplier in suppliers: 
    for material in materials: 
        c = solver.Constraint(0, int(sup_stock.loc[supplier][material]))
        for factory in factories: 
            c.SetCoefficient(orders[(factory, material, supplier)],1)
            
            
# Defining the constraints that ensure that factories order enough material to be able to manufacture all items

for factory in factories:
    for material in materials:
        c = solver.Constraint(0,solver.infinity())
        for supplier in suppliers:
            c.SetCoefficient(orders[(factory, material, supplier)],1)
            for product in products:
                c.SetCoefficient(production_volume[(factory, product)], - production_req.loc[product][material])
            

# Defining the constraints that ensure that the manufacturing capacities are not exceeded
                 
for factory in factories: 
    for product in products: 
        c = solver.Constraint(0, int(production_capacity.loc[product][factory]))  
        c.SetCoefficient(production_volume[(factory, product)],1)


In [8]:
# H.  Defining the objective function.

cost = solver.Objective()

# Material Costs  + shipping costs 
for factory in factories: 
    for supplier in suppliers:
        for material in materials:
            cost.SetCoefficient(orders[(factory, material, supplier)] , 
                                       raw_material_cost.loc[supplier][material] + raw_material_shipping.loc[supplier][factory])
            

# production cost of each factory 
for factory in factories: 
    for product in products: 
        cost.SetCoefficient(production_volume[(factory, product)], int(production_cost.loc[product][factory]))

# shipping cost to customers 
for factory in factories: 
    for customer in customers:
        for product in products: 
            cost.SetCoefficient(delivery[(factory, customer, product)], int(shipping_costs.loc[factory][customer]))


In [9]:
# I. Solving the ILP and determine the optimal overall cost
            
cost.SetMinimization()
status = solver.Solve()

if status == solver.OPTIMAL:
    print("Optimal Solution Found")
print("Optimal Overall Cost: ", solver.Objective().Value())

# printig supplier's Bill for all factories also priting  value of material delivered
print("\nSupplier Bill and order quantity")
print("****************************")
for factory in factories:
    print(factory,":")
    
    for supplier in suppliers:
        factory_cost = 0
        print("  ",supplier,":")
        for material in materials:
            print("\t",material,":", orders[(factory, material, supplier)].solution_value())
            
            factory_cost += orders[(factory, material, supplier)].solution_value() * raw_material_cost.loc[supplier][material]
            factory_cost += orders[(factory, material, supplier)].solution_value() * float(raw_material_shipping.loc[supplier][factory])
        print("  ",supplier," Bill: ", factory_cost)

#  printing manufacturing Cost for all factories
print("Production Volume:")
print("****************************")

for factory in factories:
    print(factory,":")
    production_cost_total = 0
    for product in products:
        if production_volume[(factory, product)].solution_value() >0:
            print("  ",product,": ",production_volume[(factory, product)].solution_value())
            production_cost_total += production_volume[(factory, product)].solution_value() * production_cost.loc[product][factory]
    print("   Manufacturing cost: ", production_cost_total)


Optimal Solution Found
Optimal Overall Cost:  49315.0

Supplier Bill and order quantity
****************************
Factory A :
   Supplier A :
	 Material A : 20.0
	 Material B : 20.0
	 Material C : 0.0
	 Material D : 0.0
   Supplier A  Bill:  2800.0
   Supplier B :
	 Material A : 19.0
	 Material B : 4.0
	 Material C : 0.0
	 Material D : 0.0
   Supplier B  Bill:  2155.0
   Supplier C :
	 Material A : 0.0
	 Material B : 0.0
	 Material C : 0.0
	 Material D : 0.0
   Supplier C  Bill:  0.0
   Supplier D :
	 Material A : 0.0
	 Material B : 0.0
	 Material C : 14.0
	 Material D : 50.0
   Supplier D  Bill:  6440.0
   Supplier E :
	 Material A : 0.0
	 Material B : 0.0
	 Material C : 0.0
	 Material D : 0.0
   Supplier E  Bill:  0.0
Factory B :
   Supplier A :
	 Material A : 0.0
	 Material B : 0.0
	 Material C : 0.0
	 Material D : 0.0
   Supplier A  Bill:  0.0
   Supplier B :
	 Material A : 6.0
	 Material B : 34.0
	 Material C : 0.0
	 Material D : 0.0
   Supplier B  Bill:  3500.0
   Supplier C :

In [10]:
# printing shipping Cost
print("\nShipping to Customer:") 
print("****************************")

for customer in customers:   
    shipping_cost = 0
    print(customer)
    for product in products:
        print("  ", product)
        for factory in factories: 
            print("\t",factory,": ",delivery[(factory, customer, product)].solution_value())
            shipping_cost += delivery[(factory, customer, product)].solution_value() * shipping_costs.loc[factory][customer]
    print("   Shipping Cost: ", shipping_cost)


Shipping to Customer:
****************************
Customer A
   Product A
	 Factory A :  5.0
	 Factory B :  0.0
	 Factory C :  2.0
   Product B
	 Factory A :  0.0
	 Factory B :  0.0
	 Factory C :  0.0
   Product C
	 Factory A :  0.0
	 Factory B :  0.0
	 Factory C :  0.0
   Product D
	 Factory A :  0.0
	 Factory B :  0.0
	 Factory C :  1.0
   Shipping Cost:  280.0
Customer B
   Product A
	 Factory A :  1.0
	 Factory B :  2.0
	 Factory C :  0.0
   Product B
	 Factory A :  0.0
	 Factory B :  0.0
	 Factory C :  0.0
   Product C
	 Factory A :  0.0
	 Factory B :  0.0
	 Factory C :  0.0
   Product D
	 Factory A :  0.0
	 Factory B :  0.0
	 Factory C :  0.0
   Shipping Cost:  110.0
Customer C
   Product A
	 Factory A :  0.0
	 Factory B :  0.0
	 Factory C :  0.0
   Product B
	 Factory A :  1.0
	 Factory B :  1.0
	 Factory C :  0.0
   Product C
	 Factory A :  0.0
	 Factory B :  0.0
	 Factory C :  0.0
   Product D
	 Factory A :  0.0
	 Factory B :  0.0
	 Factory C :  3.0
   Shipping Cost:  370.0


In [11]:
#  final output of next few lies is cost incured(per unit) to satisfy demand of customer for a product
print("\nMaterial Bifurcation and Cost per unit")
print("****************************")

for customer in customers:
    print(customer)
    for product in products:
        
        unit_cost_per_product = 0
        if int(customer_demand.loc[product][customer]) >0:
            print("  ", product)
            for factory in factories:
                
                if delivery[(factory, customer, product)].solution_value() >0:
                    print("\t", factory, ": ")
                    # Calculating the Shipping cost from factory to customer based on number of products
                    shipping_cost = delivery[(factory, customer, product)].solution_value() * shipping_costs.loc[factory][customer]
                    # Calculating the manufacturing cost 
                    man_cost = delivery[(factory, customer, product)].solution_value() * production_cost.loc[product][factory]
                    unit_cost_per_product += shipping_cost
                    unit_cost_per_product += man_cost
                    material_cost_to_customer = 0
                    for material in materials:
                        material_units = 0
                        material_units += delivery[(factory, customer, product)].solution_value() * production_req.loc[product][material]
                        
                        print("\t  ",material,": ", material_units)  
                        #raw material cost
                        material_cost = 0
                        #raw material cost
                        rshipping_cost = 0 
                        material_count = 0
                        for supplier in suppliers:
                            material_cost +=  orders[(factory, material, supplier)].solution_value() * raw_material_cost.loc[supplier][material]
                            rshipping_cost += orders[(factory, material, supplier)].solution_value() * raw_material_shipping.loc[supplier][factory]
                            material_count += orders[(factory, material, supplier)].solution_value()
                        material_cost_to_customer = ((material_cost + rshipping_cost)/material_count) * material_units
                        unit_cost_per_product += material_cost_to_customer
            print("\t cost per unit : ", unit_cost_per_product/int(customer_demand.loc[product][customer]))



Material Bifurcation and Cost per unit
****************************
Customer A
   Product A
	 Factory A : 
	   Material A :  25.0
	   Material B :  15.0
	   Material C :  0.0
	   Material D :  0.0
	 Factory C : 
	   Material A :  10.0
	   Material B :  6.0
	   Material C :  0.0
	   Material D :  0.0
	 cost per unit :  910.9752747252747
   Product D
	 Factory C : 
	   Material A :  3.0
	   Material B :  2.0
	   Material C :  4.0
	   Material D :  15.0
	 cost per unit :  3913.75
Customer B
   Product A
	 Factory A : 
	   Material A :  5.0
	   Material B :  3.0
	   Material C :  0.0
	   Material D :  0.0
	 Factory B : 
	   Material A :  10.0
	   Material B :  6.0
	   Material C :  0.0
	   Material D :  0.0
	 cost per unit :  745.7051282051283
Customer C
   Product B
	 Factory A : 
	   Material A :  0.0
	   Material B :  0.0
	   Material C :  2.0
	   Material D :  5.0
	 Factory B : 
	   Material A :  0.0
	   Material B :  0.0
	   Material C :  2.0
	   Material D :  5.0
	 cost per unit :  

In [12]:
# calculating total cost to factories for satisfying customer demand
print("---------------------------------------")
print("Total cost to factories and units delivered")
for factory in factories:
    print("    "+factory)
    factory_cost = 0
    volume_produced = 0
    for supplier in suppliers:
        for material in materials:
            factory_cost = factory_cost + orders[(factory,material,supplier)].solution_value()*(raw_material_shipping.loc[supplier][factory] + raw_material_cost.loc[supplier][material])
    for product in products:
        factory_cost = factory_cost + production_volume[(factory,product)].solution_value()*production_cost.loc[product][factory]
        for customer in customers:
            factory_cost = factory_cost + delivery[(factory,customer,product)].solution_value()*shipping_costs.loc[factory][customer] 
            volume_produced = volume_produced + delivery[(factory,customer,product)].solution_value()
    print("        Cost:"+str(factory_cost)+"    Units Delivered:"+str(volume_produced))      

---------------------------------------
Total cost to factories and units delivered
    Factory A
        Cost:12785.0    Units Delivered:10.0
    Factory B
        Cost:14260.0    Units Delivered:7.0
    Factory C
        Cost:22270.0    Units Delivered:7.0
